In [1]:
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.profiler import profile, record_function, ProfilerActivity
from utils import *
from fla_parallelized import FLANN, npDataset

In [2]:
X,y,weight = diabetes_data()
X,y = torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)
hidden_dims = [50,25,10]
attn_heads = 5
model = FLANN(input_dim=108, hidden_dims=hidden_dims, output_dim=1, attn_heads=attn_heads, activation=nn.ReLU)
criterion = nn.BCEWithLogitsLoss(pos_weight=weight)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [3]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("back"):
        # model(X[:64])
        loss = criterion(model(X[:64]), y[:64].unsqueeze(1)).backward()
            
            
            

STAGE:2024-07-11 17:08:34 59394:4731503 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-07-11 17:08:34 59394:4731503 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-07-11 17:08:34 59394:4731503 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [6]:
print(prof.key_averages().table(sort_by="cpu_time_total"))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   back        22.76%      17.743ms       100.00%      77.942ms      77.942ms             1  
                                              aten::bmm        18.06%      14.076ms        21.54%      16.786ms       1.865ms             9  
      autograd::engine::evaluate_function: BmmBackward0         0.32%     246.000us        16.36%      12.750ms       4.250ms             3  
                                           BmmBackward0         0.03%      24.000us        16.04%      12.504ms       4.168ms             3  
autogr

In [8]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total"))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------------------------------------------------------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                        Input Shapes  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------------------------------------------------------------  
                                                   back        22.76%      17.743ms       100.00%      77.942ms      77.942ms             1                                                                  []  
      autograd::engine::evaluate_function: BmmBackward0         0.32%     246.000us        16.36%      12.750ms       4.250ms             3                     